# Лабораторная работа 4. Вариант 7.

## Задание
Найти приближенное решение интегрального уравнения $u(x) - \int_{a}^{b}H(x,y)u(y)\,dy=f(x)$, используя одну из квадратурных формул:

1) Составная формула трапеций.

2) Составная формула средних прямоугольников.

3) Составная формула Симпсона.

4) Формулы Гаусса с 2, 3, 4 и т.д. узлами.

5) Составная формула Гаусса с двумя узлами.

6) Составная формула Гаусса с тремя узлами.

Количество разбиений в составных формулах рекомендуется удваивать до 
тех пор, пока значения приближенных решений в точках $a, (a+b)/2, b$ не 
будут совпадать с точностью $\varepsilon$.

In [1]:
import numpy as np
from numpy import linalg as LA
from tabulate import tabulate


A = 0
B = 1
N = 1
EPSILON = 0.001


def find_solution_gauss(A, b, use_pivoting=False):
    n = len(A)
    if b.size != n:
        raise ValueError('Incompatible sizes of A and b')

    for i in range(n - 1):
        if use_pivoting:
            pivot_index = abs(A[i:, i]).argmax() + i
            if A[pivot_index, i] == 0:
                raise ValueError('A is singular')

            if pivot_index != i:
                A[[i, pivot_index]] = A[[pivot_index, i]]
                b[[i, pivot_index]] = b[[pivot_index, i]]
        else:
            if A[i, i] == 0:
                raise ValueError('Pivot is zero')

        for j in range(i + 1, n):
            ratio = A[j, i] / A[i, i]
            A[j, i:] -= ratio * A[i, i:]
            b[j] -= ratio * b[i]

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

    return x

### Уравнение
### $u(x) - 0.5\int_{0}^{1}th(xy)u(y)\,dy = x - 0.5$
### Метод
Составная формула Симпсона

In [2]:
def f(x):
    return x - 0.5


def H(x, y):
    return np.tanh(x * y)


def get_points(a, b, f, n, x):
    step = np.abs(b - a) / n
    points = []
    for i in range(n):
        x_i = a + i * step
        f_x = H(x, x_i)
        points.append((x, f_x))
        
    return points


def find_simpson_coefs(points):
    a = points[0][0]
    n = len(points)
    b = points[-1][0]
    h = np.abs(b - a) / n
    coefs = []
    k_out = (b - a) / (6 * n)
    
    coefs.append(k_out * points[0][1])
    
    for i in range(1, len(points) - 1):
        k_in = 2 if i % 2 == 0 else 4
        coefs.append(k_out * k_in * points[i][1])
        
    coefs.append(k_out * points[-1][1])
    return coefs


def find_u_n(a, b, h, f, n, x, print_tmp_res=False):
    n *= 2
    if print_tmp_res:
        print(f'Кол-во узлов: {n}')
    points = get_points(a, b, f, n, x)
    if print_tmp_res:
        print(f'Узлы: {points}')
    coefs = find_simpson_coefs(points)
    if print_tmp_res:
        print(f'Коэф-ты: {coefs}')
    
    D = np.zeros((len(coefs), len(coefs)))
    for i in range(n):
        for j in range(n):
            delta = 1 if i == j else 0
            D[i, j] = delta - coefs[j] * h(points[i][0], points[j][0])
    g = np.array([coefs]).T
    if print_tmp_res and n < 5:
        print(f'D: {D}')
        print(f'g: {g}')
        
    z = find_solution_gauss(D, g)
    
    u_n = 0
    
    for i in range(n):
        u_n += coefs[i] * h(x, points[i][0]) * z[i]
        
    u_n = -0.5 * u_n + f(x)
    return u_n

In [3]:
def find_max_diff(prev_values, values):
    return max([np.abs(values[i] - prev_values[i]) for i in range(len(values))])


def print_table(a, b, h, f, n, print_tmp_res=False):
    accurate = False
    rows = []
    u_n_a = find_u_n(a, b, h, f, n, a, print_tmp_res)
    u_n_ab = find_u_n(a, b, h, f, n, (a + b) / 2, print_tmp_res)
    u_n_b = find_u_n(a, b, h, f, n, b, print_tmp_res)
    
    rows.append([f'u^{n * 2}', str(u_n_a), str(u_n_ab), str(u_n_b)])
    
    while not accurate:
        n += 1
        new_u_n_a = find_u_n(a, b, h, f, n, a, print_tmp_res)
        new_u_n_ab = find_u_n(a, b, h, f, n, (a + b) / 2, print_tmp_res)
        new_u_n_b = find_u_n(a, b, h, f, n, b, print_tmp_res)
        rows.append([f'u^{n * 2} (x)', str(new_u_n_a), str(new_u_n_ab), str(new_u_n_b)])
        error = find_max_diff([u_n_a, u_n_ab, u_n_b], [new_u_n_a, new_u_n_ab, new_u_n_b])
        accurate = error < EPSILON
    
    rows.append([f'u^{n} (x) - u^{n / 2} (x)', str(new_u_n_a - u_n_a), str(new_u_n_ab - u_n_ab), str(new_u_n_b - u_n_b)])
    rows.append(['Решение, полученное в 1-ом методе', rows[-2], rows[-2], rows[-2]])
    
    print(tabulate(rows, headers=['x', 'a', '(a+b)/2', 'b']))
    print(f'Оценка, полученная в 1-ом методе: {error}')

print_table(A, B, H, f, N, True)

Кол-во узлов: 2
Узлы: [(0, 0.0), (0, 0.0)]
Коэф-ты: [0.0, 0.0]
D: [[1. 0.]
 [0. 1.]]
g: [[0.]
 [0.]]
Кол-во узлов: 2
Узлы: [(0.5, 0.0), (0.5, 0.24491866240370913)]
Коэф-ты: [0.0, 0.0]
D: [[1. 0.]
 [0. 1.]]
g: [[0.]
 [0.]]
Кол-во узлов: 2
Узлы: [(1, 0.0), (1, 0.46211715726000974)]
Коэф-ты: [0.0, 0.0]
D: [[1. 0.]
 [0. 1.]]
g: [[0.]
 [0.]]
Кол-во узлов: 4
Узлы: [(0, 0.0), (0, 0.0), (0, 0.0), (0, 0.0)]
Коэф-ты: [0.0, 0.0, 0.0, 0.0]
D: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
g: [[0.]
 [0.]
 [0.]
 [0.]]
Кол-во узлов: 4
Узлы: [(0.5, 0.0), (0.5, 0.12435300177159621), (0.5, 0.24491866240370913), (0.5, 0.35835739835078595)]
Коэф-ты: [0.0, 0.0, 0.0, 0.0]
D: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
g: [[0.]
 [0.]
 [0.]
 [0.]]
Кол-во узлов: 4
Узлы: [(1, 0.0), (1, 0.24491866240370913), (1, 0.46211715726000974), (1, 0.6351489523872873)]
Коэф-ты: [0.0, 0.0, 0.0, 0.0]
D: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
g: [[0.]
 [0.]
 [0.]
 [0.]]
x  

In [4]:
#print_table(A, B, H, f, N, True)